In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.settings import *
import pickle as pkl
import gzip
from src.tools.vis_utils import VisualizationConfig, FIGSIZE, visualize_dendrogram, plot_hexagons_map, plot_clusters, ensure_geometry_type
from src.tools.dim_reduction import reduce_tsne
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import dataclasses
import json5 as json
from scipy.cluster.hierarchy import cut_tree
from src.tools.configs import ExperimentConfig
from src.tools.clustering import remap_cluster
from src.tools.feature_extraction import features_wide_to_long
from sklearn.metrics import pairwise_distances, pairwise_distances_argmin
from tqdm.auto import tqdm
import seaborn as sns
import plotly.express as px
import numpy as np
from IPython.display import display
import contextily as ctx
import operator
from src.tools.vis_utils import visualize_kepler, save_config


In [ ]:
run_name = "run_01"
run_dir = RUNS_DATA_DIR / run_name

In [ ]:
with open(run_dir / "experiment_config.json", "r") as f:
    ec_json = json.load(f)
    ec = ExperimentConfig(**ec_json)
ec

In [ ]:
vc = VisualizationConfig(
    n_clusters=None,
    distance_threshold=0,
    affinity="euclidean",
    linkage="ward",
    truncate_mode="level",
    p=3,
    clusters=[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    cities_to_plot=["Wrocław", "Kraków"],
    countries_subsample=["Poland", "Germany"],
    umap_n_components=2,
    umap_n_neighbours=30,
    umap_metric="euclidean",
    tsne_perplexity=100,
)

vis_dir = run_dir / "vis"
vis_dir.mkdir(parents=True, exist_ok=True)

with open(vis_dir / "vis_config.json", "w") as f:
    json.dump(dataclasses.asdict(vc), f, indent=2, quote_keys=True, trailing_commas=False)

In [ ]:
with gzip.open(run_dir / "dataset.pkl.gz", "rb") as f:
    ds = pkl.load(f)

z_df = pd.read_pickle(run_dir / "embeddings.pkl.gz")
input_df = pd.read_pickle(run_dir / "input.pkl.gz")

if ec.mode == "edges":
    z_df_edges = z_df
    z_df = z_df.groupby(level=[0, 1, 2, 3]).mean()

    input_df_edges = input_df
    input_df = input_df.groupby(level=[0, 1, 2, 3]).sum()
    input_df_mean = input_df_edges.groupby(level=[0, 1, 2, 3]).mean()

z_df_scaled = pd.DataFrame(StandardScaler().fit_transform(z_df), index=z_df.index, columns=z_df.columns)
z_df_scaled_cosine = pd.DataFrame(StandardScaler(with_std=False).fit_transform(z_df), index=z_df.index, columns=z_df.columns)
hexagons = ds.hexagons

In [ ]:
ds.__annotations__

In [ ]:
countries_subsample = vc.countries_subsample

z_df = z_df.loc[(slice(None), countries_subsample), :]
z_df_scaled = z_df_scaled.loc[(slice(None), countries_subsample), :]
z_df_scaled_cosine = z_df_scaled_cosine.loc[(slice(None), countries_subsample), :]
z_df_edges = z_df_edges.loc[(slice(None), countries_subsample), :]
ds.edges = ds.edges.loc[(slice(None), countries_subsample), :]
ds.edges_feature_selected = ds.edges_feature_selected.loc[(slice(None), countries_subsample), :]
input_df = input_df.loc[(slice(None), countries_subsample), :]
input_df_edges = input_df_edges.loc[(slice(None), countries_subsample), :]
input_df_mean = input_df_mean.loc[(slice(None), countries_subsample), :]
ds.hexagons = ds.hexagons.loc[(slice(None), countries_subsample), :]

In [ ]:
unique_cities_df = z_df.reset_index(level=[0, 1, 3], drop=True).index.unique()
print("Cities:", unique_cities_df.size)
unique_cities_df

In [ ]:
z_df

In [ ]:
feature_keys = list(ds.config.featureset_selection["features"].keys())
feature_keys

In [ ]:
edges_fs_long = features_wide_to_long(ds.edges_feature_selected.assign(geometry=ds.edges["geometry"]), feature_keys)
edges_fs_long

In [ ]:
input_df_profile = input_df.copy()
for f_k in feature_keys:
        features_for_key = [x for x in input_df.columns if f_k in x]
        input_df_profile[features_for_key] = input_df_profile[features_for_key].apply(lambda x: x / x.sum(), axis=1)

input_df_profile

In [ ]:
def get_hex_profile(hex_id: str) -> pd.Series:
    hex_profile = input_df_profile.loc[(slice(None), slice(None), slice(None), hex_id)] \
        .iloc[0] \
        .sort_values(ascending=False)

    return hex_profile[hex_profile > 0]

In [ ]:
ac_model = AgglomerativeClustering(n_clusters=vc.n_clusters, distance_threshold=vc.distance_threshold, affinity=vc.affinity, linkage=vc.linkage)
ac_model.fit(z_df_scaled)
# ac_model = AgglomerativeClustering(n_clusters=vc.n_clusters, distance_threshold=vc.distance_threshold, affinity="cosine", linkage="average")  # use with_std=False in StandardScaler
# ac_model.fit(z_df_scaled_cosine)

In [ ]:
dendrogram_path = vis_dir / "dendrogram.png"

fig, ax = plt.subplots(figsize=(12, 7))
plt.xlabel("Number of microregions")

linkage_matrix = visualize_dendrogram(ac_model, truncate_mode=vc.truncate_mode, p=vc.p, ax=ax)

plt.tight_layout()
fig.savefig(dendrogram_path, facecolor='w')
plt.show()

In [ ]:
z_df_with_clusters = z_df.copy()
cut_tree_results = cut_tree(linkage_matrix, n_clusters = vc.clusters)
clusters_divided = [None]
for index, c in tqdm(list(enumerate(vc.clusters))):
    assigned_clusters = cut_tree_results[:, index]
    z_df_with_clusters[f"cluster_{c}"] = pd.Series(assigned_clusters, index=z_df.index).astype("category")

    if index > 0:
        remapped_clusters, cluster_divided_id = remap_cluster(z_df_with_clusters, c=c)
        clusters_divided.append(cluster_divided_id)
        z_df_with_clusters[f"cluster_{c}"] = remapped_clusters

hexagons_clustered = hexagons.join(z_df_with_clusters[[f"cluster_{c}" for c in vc.clusters]]).dropna().set_crs(epsg=4326)

In [ ]:
df_clusters = z_df_with_clusters[[f"cluster_{c}" for c in vc.clusters]]
df_clusters.to_pickle(vis_dir / "clusters.pkl.gz")

import csv
with open(vis_dir / "clusters_divided.csv", "w") as f:
    write = csv.writer(f)
    write.writerow(clusters_divided)

In [ ]:
edges_keplergl = edges_fs_long.astype(str)

In [ ]:
config_name = "edges_hexes"

hexagons_keplergl = hexagons_clustered.reset_index().drop(columns=["coordinates", "parent", "children"])
hexagons_keplergl["h3_id"] = hexagons_keplergl["h3_id"].map(lambda x: f"hex_{x}")


m = visualize_kepler(data={
        "edges": edges_keplergl,
        "hexagons": hexagons_keplergl,
    }, 
    config_name=config_name)
m

In [ ]:
save_config(m, config_name=config_name)

In [ ]:
# from src.tools.vis_utils import save_kepler_map
# save_kepler_map(m, config_name)

In [ ]:
hexagons_dir = vis_dir / "hexagons"
hexagons_dir.mkdir(parents=True, exist_ok=True)
for ctp in tqdm(vc.cities_to_plot):
    for idx, c in enumerate(vc.clusters):
        cluster_divided_id = clusters_divided[idx]
        ax = plot_hexagons_map(hexagons_clustered.loc[:, :, ctp], ds.edges.loc[:, :, ctp], f"cluster_{c}", title=f"Division on cluster {cluster_divided_id} to {cluster_divided_id} and {c - 1}" if cluster_divided_id is not None else "Initial division")
        ax.set_axis_off()
        plt.tight_layout()
        plt.savefig(hexagons_dir / f"{ctp}_cluster_{c}.png", facecolor='w', dpi=100)
        plt.close()
        print(f"Division on cluster {cluster_divided_id} to {cluster_divided_id} and {c - 1}" if cluster_divided_id is not None else "Initial division")

In [ ]:
z_df_tsned = reduce_tsne(z_df_scaled, n_components=vc.umap_n_components, perplexity=vc.tsne_perplexity)

In [ ]:
tsne_dir = vis_dir / "tsne"
tsne_dir.mkdir(parents=True, exist_ok=True)
for idx, c in enumerate(vc.clusters):
    cluster_divided_id = clusters_divided[idx]
    cluster_to_show = f"cluster_{c}"
    z_df_tsned["cluster"] = z_df_with_clusters[cluster_to_show]
    fig = plot_clusters(z_df_tsned.sort_values("cluster"), title=f"Division on cluster {cluster_divided_id} to {cluster_divided_id} and {c - 1}" if cluster_divided_id is not None else "Initial division")
    # fig = plot_clusters(z_df_tsned.sort_values("cluster"), title="")
    fig.write_image(tsne_dir / f"tsne_hexes_{c}.png")
    fig.show()

In [ ]:
def hex_diff(z_df, hex_id_1, hex_id_2, operator_func, metric, city=None, top_n=5):
    hex_1_z = z_df.loc[(slice(None), slice(None), slice(None), hex_id_1), :]
    hex_2_z = z_df.loc[(slice(None), slice(None), slice(None), hex_id_2), :]
    hex_diff_z = operator_func(hex_1_z.values, hex_2_z.values).reshape(1, -1)

    if city is None:
        df = z_df
    else: 
        df = z_df.loc[(slice(None), slice(None), city), :]


    hex_diff_closest_ids = pairwise_distances(df, hex_diff_z, metric=metric).argsort(axis=0)[:top_n].squeeze()
    hex_diff_closest = df.iloc[np.array([hex_diff_closest_ids]).reshape(-1,)]

    hex_1 = input_df.loc[hex_1_z.index].assign(diff="first").assign(top_n=0)
    hex_2 = input_df.loc[hex_2_z.index].assign(diff="second").assign(top_n=0)
    hex_diff = input_df.loc[hex_diff_closest.index].assign(diff="diff").assign(top_n=list(range(top_n))).drop(hex_1.index, errors="ignore").drop(hex_2.index, errors="ignore")

    diff_gdf = gpd.GeoDataFrame(pd.concat([hex_1, hex_2, hex_diff], axis=0).join(hexagons["geometry"]), crs="EPSG:4326")

    return diff_gdf

def hex_interp(z_df, hex_start_id, hex_end_id, metric, city=None, n_steps=20):
    z_start = z_df.loc[(slice(None), slice(None), slice(None), hex_start_id), :]
    z_end = z_df.loc[(slice(None), slice(None), slice(None), hex_end_id), :]
    hex_start = input_df.loc[z_start.index].assign(type="start").assign(step=0)
    hex_end = input_df.loc[z_end.index].assign(type="end").assign(step=0)


    if city is None:
        df = z_df
    else: 
        df = z_df.loc[(slice(None), slice(None), city), :]
    steps = np.linspace(z_start, z_end, n_steps)
    hexes_steps = pd.DataFrame()
    for idx, step in enumerate(steps): 

        hex_diff_closest_id = pairwise_distances_argmin(df, step, axis=0, metric=metric).item()
        hex_diff_closest = df.iloc[[hex_diff_closest_id]]

        
        hex_step = input_df.loc[hex_diff_closest.index]
        hexes_steps = pd.concat([hexes_steps, hex_step], axis=0)
    hexes_steps = hexes_steps.drop_duplicates().drop(hex_start.index, errors="ignore").drop(hex_end.index, errors="ignore")
    hexes_steps = hexes_steps.assign(type="diff").assign(step=list(range(hexes_steps.shape[0])))

    steps_final = pd.concat([hex_start, hexes_steps, hex_end], axis=0)
    steps_final = features_wide_to_long(steps_final, feature_keys)
    steps_final = gpd.GeoDataFrame(steps_final.join(hexagons["geometry"]), crs="EPSG:4326")
    return steps_final


vis_operations_dir = vis_dir / "operations"
vis_operations_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
def plot_hex_profile(hex_profile: pd.Series) -> plt.Axes:
    ax = hex_profile.plot(kind="bar", figsize=(8, 5))
    plt.xticks(rotation=60)
    plt.xlabel("Tag")
    plt.ylabel("Share")
    plt.tight_layout()
    return ax


hex_profile_ids = ["881e2055a5fffff", "881e2042d9fffff"]
for hex_profile_id in hex_profile_ids:
    hex_profile = get_hex_profile(hex_profile_id)
    plot_hex_profile(hex_profile)
    plt.savefig(vis_operations_dir / f"hex_{hex_profile_id}_profile.png", facecolor='w')
    plt.show()


In [ ]:
hex_id_first = "881e2055a5fffff"
hex_id_second = "881e2042d9fffff"
operator_func = operator.add

top_n = 1
city = "Wrocław"

diff_gdf = hex_diff(z_df_scaled_cosine, hex_id_first, hex_id_second, operator_func, metric="cosine", city=city, top_n=top_n)
# diff_gdf = hex_diff(z_df_scaled, hex_id_first, hex_id_second, operator_func, metric="euclidean", city=city, top_n=top_n)
unique_cities_in_diff = diff_gdf.index.droplevel(3).unique()
display(unique_cities_in_diff)

diff_gdf = diff_gdf.reset_index()
diff_gdf["h3_id"] = diff_gdf["h3_id"].map(lambda x: f"hex_{x}")
config_name = "hex_diff"
m = visualize_kepler(data={
        "edges": edges_keplergl.droplevel(3).loc[unique_cities_in_diff],
        "diff": diff_gdf.copy(),
    }, 
    config_name=config_name)
m

In [ ]:
save_config(m, config_name=config_name)

In [ ]:
start_h3_id = "881e2055a5fffff"
end_h3_id = "881e2042d9fffff"

city = "Wrocław"
n_steps = 5

metric = "euclidean"
df = z_df_scaled

# metric = "cosine"
# df = z_df_scaled_cosine

steps_final = hex_interp(df, start_h3_id, end_h3_id, metric=metric, city=city, n_steps=n_steps)

unique_cities_in_steps = steps_final.index.droplevel(3).unique()
display(unique_cities_in_steps)
display(steps_final)
steps_final = steps_final.reset_index()
steps_final["h3_id"] = steps_final["h3_id"].map(lambda x: f"hex_{x}")
config_name = "hex_interp"
m = visualize_kepler(data={
        "edges": edges_keplergl.droplevel(3).loc[unique_cities_in_steps],
        "diff": steps_final.copy(),
    }, 
    config_name=config_name)
m

In [ ]:
save_config(m, config_name=config_name)

In [ ]:
def cluster_difference(df_mean, first_cluster, second_cluster):
    return (df_mean.loc[first_cluster] - df_mean.loc[second_cluster]).sort_values(ascending=False)

vis_features_dir = vis_dir / "features"

for idx, c in tqdm(list(enumerate(vc.clusters))):
    c_name = f"cluster_{c}"
    input_df_cluster_mean = input_df_mean.groupby(df_clusters[c_name]).mean()


    # input_df_cluster_mean_perc = input_df_cluster_mean.apply(lambda x: x / x.sum())  # or
    input_df_cluster_mean_perc = input_df_cluster_mean.copy()
    for f_k in feature_keys:
        features_for_key = [x for x in input_df_cluster_mean_perc.columns if f_k in x]
        input_df_cluster_mean_perc[features_for_key] = input_df_cluster_mean_perc[features_for_key].apply(lambda x: x / x.sum(), axis=1)


    vis_features_cluster_dir = vis_features_dir / c_name
    vis_features_cluster_dir.mkdir(parents=True, exist_ok=True)


    features_wide_to_long(input_df_cluster_mean_perc, feature_keys).T.to_csv(vis_features_cluster_dir / "clusters_characteristics.csv")


    cluster_divided_id = clusters_divided[idx]
    if cluster_divided_id is None:
        c_first = 1
        c_second = 0
    elif cluster_divided_id != -1:
        c_first = c - 1
        c_second = cluster_divided_id
    else:
        c_first, c_second = None, None

    if c_first not in (None, -1) and c_second not in (None, -1):
        mean_cluster_difference = cluster_difference(input_df_cluster_mean, c_first, c_second)
        mean_cluster_difference = mean_cluster_difference[mean_cluster_difference.abs() >= 0.01]
        mean_cluster_difference.plot(kind="bar", figsize=(10, 6), color=(mean_cluster_difference >= 0).map({True: "green", False: "red"}), title=f"Cluster difference: {c_first} - {c_second}")
        plt.ylabel("Difference in share")
        plt.tight_layout()
        plt.savefig(vis_features_cluster_dir / f"cluster_difference_{c_first}-{c_second}.png", facecolor='w')
        plt.close()
        print(f"Cluster difference: {c_first} - {c_second}")


    for f_k in feature_keys:
        features_for_key = [x for x in input_df_cluster_mean_perc.columns if f_k in x]
        fig, ax = plt.subplots(figsize=(10, 5))
        df = input_df_cluster_mean_perc[features_for_key]
        df.columns = df.columns.map(lambda x: x.split("_", 1)[1])
        df.plot(kind="bar", stacked=True, ax=ax, cmap="tab20")
        ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
        plt.xlabel("Cluster")
        plt.ylabel("Share")
        plt.xticks(rotation=0)
        plt.tight_layout()
        plt.savefig(vis_features_cluster_dir / f"{f_k}.png", facecolor='w')
        plt.close()
    

In [ ]:
pca_rgb_dir = vis_dir / "pca_rgb"
pca_rgb_dir.mkdir(parents=True, exist_ok=True)

z_df_rgb = pd.DataFrame(PCA(n_components=3).fit_transform(z_df)).set_index(z_df.index)
z_df_rgb.columns = ["r", "g", "b"]
z_df_rgb_scaled = pd.DataFrame(MinMaxScaler().fit_transform(z_df_rgb)).set_index(z_df.index)
rgb_gdf = gpd.GeoDataFrame(z_df_rgb_scaled.join(hexagons[["geometry"]]), crs="EPSG:4326")
for ctp in tqdm(vc.cities_to_plot):
    fig, ax = plt.subplots(figsize=(10, 9))
    ax.set_aspect('equal')
    ax.set_title(f"{ctp} RGB")
    ax.set_axis_off()
    gpd_rgb = rgb_gdf.loc[:, :, ctp]
    gpd_rgb.to_crs(epsg=3857).plot(ax=ax, alpha=0.7, color=gpd_rgb[[0, 1, 2]].to_numpy())
    plt.tight_layout()
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
    fig.savefig(pca_rgb_dir / f"{ctp}.png", facecolor='w', dpi=100)
    plt.close()